In [10]:
# Carrega todas as colunas e garante uma chave estável

import pandas as pd

df = pd.read_csv("../../data/raw/tccs.csv")

assert "resumo" in df.columns, "A coluna 'resumo' não existe no CSV."

# DOC_ID estável para cruzamentos
if "DOC_ID" not in df.columns:
    df.insert(0, "DOC_ID", range(len(df)))

print("Shape:", df.shape, "| colunas (10 primeiras):", list(df.columns)[:10], "...")

Shape: (423, 7) | colunas (10 primeiras): ['DOC_ID', 'ano', 'titulo', 'autor', 'orientador', 'resumo', 'url'] ...


In [11]:
# Configuração spaCy + stopwords (apenas nativas do spaCy)

import spacy, unidecode
from spacy.lang.pt.stop_words import STOP_WORDS

# Modelo PT do spaCy
nlp = spacy.load("pt_core_news_sm", disable=["ner", "parser"])

# Normaliza as stopwords para o mesmo espaço dos lemas
stopwords_norm = {unidecode.unidecode(w.lower()) for w in STOP_WORDS}
print("Stopwords (spaCy) normalizadas:", len(stopwords_norm))

Stopwords (spaCy) normalizadas: 403


In [12]:
# Funções de normalização e lematização (preserva acentos até lematizar)

def normalize_text_lower(s: str) -> str:
    return s.lower()

def tokenize_lemmatize(text: str):
    doc = nlp(text)
    toks = []
    for t in doc:
        if not t.is_alpha:
            continue
        lemma_str = unidecode.unidecode(t.lemma_.lower()).strip()
        for sub in lemma_str.split():
            if len(sub) <= 2 or sub in stopwords_norm:
                continue
            toks.append(sub)
    return toks


In [13]:
# Aplicação nos resumos

resumos_lower = df["resumo"].fillna("").astype(str).map(normalize_text_lower)
docs_tokens = [tokenize_lemmatize(t) for t in resumos_lower]

print("Exemplo original:\n", df["resumo"].iloc[0])
print("\nApós processamento (30 primeiros tokens):\n", docs_tokens[0][:30])

Exemplo original:
 Gerenciar a agenda e fornecer informações ao cliente são tarefas que oneram muito o tempo do pequeno empreendedor, que, em muitos casos, gerencia e opera um negócio sozinho, precisando maximizar seu tempo produzindo para obter melhores resultados e manter o seu empreendimento em atividade. Este trabalho tem como objetivo propor o desenvolvimento de um sistema web que oferece ao empreendedor um meio de fornecer as principais informações sobre seu negócio e visualizar os serviços agendados. O sistema provê ao consumidor dos serviços prestados uma forma de visualizar as informações sobre um determinado serviço e efetuar o agendamento em horário disponível e conveniente para si, dispensando a necessidade de qualquer tipo de contato prévio com o prestador do serviço. A solução foi avaliada por 23 usuários, obtendo um índice de satisfação médio de 4.7 entre os dois grupos de usuários (prestadores de serviços e consumidores), constatando ser uma proposta eficaz na solução d

In [14]:
# Bigramas com Gensim Phrases

from gensim.models.phrases import Phrases, Phraser

MIN_COUNT = 5
THRESHOLD = 10.0

phrases = Phrases(docs_tokens, min_count=MIN_COUNT, threshold=THRESHOLD, delimiter="_")
bigram_phraser = Phraser(phrases)

# Aplica o detector de bigramas
docs_tokens = [bigram_phraser[doc] for doc in docs_tokens]
print("Bigramas aplicados.")

Bigramas aplicados.


In [15]:
# Checagem dos n-gramas aprendidos

# Inspeciona quantos tokens com "_" surgiram e mostra exemplos
tokens_flat = [tok for doc in docs_tokens for tok in doc]
ngrams = [t for t in tokens_flat if "_" in t]
print("Total de tokens:", len(tokens_flat))
print("Total de n-gramas (com '_'):", len(ngrams))
print("Exemplos:", ngrams[:20])


Total de tokens: 32892
Total de n-gramas (com '_'): 1334
Exemplos: ['fornecer_informacao', 'trabalho_objetivo', 'desafio_significativo', 'aprendizado_maquina', 'contribuir_avanco', 'rede_neural', 'resultado_indicar', 'presente_trabalho', 'tomar_decisao', 'estudo_investigar', 'modelo_linguagem', 'linguagem_programacao', 'desenvolvimento_software', 'desenvolvimento_software', 'processo_desenvolvimento', 'fornecer_informacao', 'processo_desenvolvimento', 'desafio_significativo', 'isolamento_social', 'apresentar_solucao']


In [16]:
# Salva o modelo de bigramas para reuso

import os
BIGRAM_PATH = "../../data/interim/lda/bigram_phraser.pkl"
os.makedirs(os.path.dirname(BIGRAM_PATH), exist_ok=True)
bigram_phraser.save(BIGRAM_PATH)
print("Bigram phraser salvo em:", BIGRAM_PATH)

Bigram phraser salvo em: ../../data/interim/lda/bigram_phraser.pkl


In [17]:
# Salvando resultado (mantém todas as colunas + RESUMO_PREP)

import os, pickle

df["RESUMO_PREP"] = [" ".join(t) for t in docs_tokens]

OUT_CSV = "../../data/interim/lda/prep.csv"
OUT_TOKENS = "../../data/interim/lda/tokens.pkl"

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

df.to_csv(OUT_CSV, index=False)

with open(OUT_TOKENS, "wb") as f:
    pickle.dump(docs_tokens, f)

print("OK ->", OUT_CSV, "| docs:", len(df), "| colunas:", len(df.columns))
print("Tokens salvos em:", OUT_TOKENS, "| docs_tokens:", len(docs_tokens))

OK -> ../../data/interim/lda/prep.csv | docs: 423 | colunas: 8
Tokens salvos em: ../../data/interim/lda/tokens.pkl | docs_tokens: 423


In [18]:
# Checagem de sanidade

from collections import Counter

lens = [len(d) for d in docs_tokens]
print(f"Docs: {len(docs_tokens)} | tokens/doc (min/med/ max): {min(lens)}/{sorted(lens)[len(lens)//2]}/{max(lens)}")
vocab = Counter(tok for d in docs_tokens for tok in d)
print("Top 15 tokens:", [w for w,_ in vocab.most_common(15)])

Docs: 423 | tokens/doc (min/med/ max): 0/79/168
Top 15 tokens: ['dado', 'utilizar', 'trabalho', 'usuario', 'ferramenta', 'modelo', 'uso', 'aplicacao', 'estudo', 'projeto', 'objetivo', 'resultado', 'realizar', 'desenvolvimento', 'informacao']
